### See how much your friends have been out climbing since a defined date

In [1]:
# Install Packages
from IPython.display import clear_output

In [2]:
# Import Packages
from IPython.display import clear_output
import requests
import os
import csv
import pandas as pd
from datetime import datetime, timedelta
import time

In [3]:
# Create array of input csv to be analyzed
FriendsList = []
FriendsCsvFiles = []

FriendsListPath = "./MPcsv/Climbing_Friends.csv"
# Either redownload MP Data or change this to date when you last downloaded from MP
today = datetime.today().strftime("%m-%d-%Y")

with open(FriendsListPath, "r", encoding="utf-8") as file:
    reader = csv.reader(file)
    for row in reader:
        FriendsList.append(row)

#Input csv files to be analyzed
for friend in FriendsList[1:]:
    FriendsCsvFiles.append('./MPcsv/' + friend[0] + "-" + today + ".csv")
                           
# print(FriendsList)
# print(FriendsCsvFiles)

In [4]:
def check_csv_for_date(input_files, output_file, lastClimbedDate):
    """
    Python function which goes through N number of csv files and looks at the first column of each file 
    which contains dates, check if those dates are after an entered date. If they are, then save the 
    rows in which the condition is satisfied
    
    Parameters:
        input_files (list of str): List of file paths to the input CSV files.
        output_file (str): Path to the output CSV file where filtered rows will be saved.
        date_threshold (str): The date threshold in 'YYYY-MM-DD' format. Rows with dates after this will be saved.

    Returns:
        None
    """
    try:
        # Convert the date_threshold to a datetime object
        lastClimbedDate = datetime.strptime(lastClimbedDate, "%m-%d-%Y")
        hasClimbed = []
        routesClimbed = []
        pitchesClimbed = []
        date_series = []
        date_objects = []
        daysOut = []
        # Initialize an empty DataFrame to store filtered rows
        filtered_rows = pd.DataFrame()
        i=0
        for file in input_files:
            # Read the CSV file
            i+=1
            try:
                df = pd.read_csv(file)

                # Ensure the first column contains dates
                if df.empty or len(df.columns) == 0:
                    print(f"Skipping empty or malformed file: {file}")
                    continue
                
                # Convert the first column to datetime
                df[df.columns[0]] = pd.to_datetime(df[df.columns[0]], errors='coerce')

                # Filter rows based on the date condition
                filtered_df = df[df[df.columns[0]] > lastClimbedDate]

                # Append filtered rows to the result DataFrame
                if not filtered_df.empty:
                    hasClimbed.append(FriendsList[i][0])
                    routesClimbed.append(len(filtered_df))
                    #df ask for pitches and then sum total for each person
                    pitchesClimbed.append(sum(filtered_df['Pitches'].tolist()))
                    # df ask for days and then calc unique days
                    date_series = filtered_df['Date'].tolist()
                    date_objects = [date.strftime("%Y-%m-%d") for date in date_series]
                    unique_days = set(date_objects)    
                    daysOut.append(len(unique_days))
                filtered_rows = pd.concat([filtered_rows, filtered_df], ignore_index=True)
                
            except Exception as e:
                print(f"Error processing file {file}: {e}")

        # Save the filtered rows to the output file
        if not filtered_rows.empty:
            filtered_rows.to_csv('./MPcsv/' + output_file, index=False)
            print(f"Filtered rows saved to {output_file}")
        else:
            print("No rows matched the condition.")
        
        print("\nSince ",lastClimbedDate.strftime("%m-%d-%Y"), "following friends have climbed: ")
        print(
            f"{'Friend':<15}{'Routes':>8}{'Pitches':>11}{'Days Out':>11}"
        )
        print('-' * 50)
        for name, routes, pitches, days in zip(hasClimbed,routesClimbed,pitchesClimbed, daysOut):
            print(
                f"{name:<15}{routes:>5}{pitches:>11}{days:>11}"
            )
    except Exception as e:
        print(f"An error occurred: {e}")



### Over what time period do you want to look over

In [5]:
last_week = datetime.today() - timedelta(weeks=1)
last_fortnite = datetime.today() - timedelta(weeks=2)
last_month = datetime.today() - timedelta(weeks=4)
last_halfyear = datetime.today() - timedelta(days=183)
last_year = datetime.today() - timedelta(days=365)
last_week = last_week.strftime("%m-%d-%Y")
last_fortnite = last_fortnite.strftime("%m-%d-%Y")
last_month = last_month.strftime("%m-%d-%Y")
last_halfyear = last_halfyear.strftime("%m-%d-%Y")
last_year = last_year.strftime("%m-%d-%Y")

print(
    f"{'Last week: ':<15}{last_week:>15}",
    f"{'Two weeks ago: ':<15}{last_fortnite:>15}",
    f"{'Last Month: ':<15}{last_month:>15}",
    f"{'Last 6 Months: ':<15}{last_halfyear:>15}",
    f"{'Last Year: ':<15}{last_year:>15}",
    sep="\n"
)

Last week:          01-11-2025
Two weeks ago:      01-04-2025
Last Month:         12-21-2024
Last 6 Months:      07-19-2024
Last Year:          01-19-2024


In [6]:
lastClimbedDate = "1-1-2020"

In [7]:
# Example usage
output_file = 'RecentFriendRoutes.csv'
date_threshold = lastClimbedDate
check_csv_for_date(FriendsCsvFiles, output_file, date_threshold)

Filtered rows saved to RecentFriendRoutes.csv

Since  01-01-2020 following friends have climbed: 
Friend           Routes    Pitches   Days Out
--------------------------------------------------
Eric             237        413         84
Link             586        992        188
Jack             209        336         70
Susan            181        265         58
Oak             1306       2077        345
Annie           1239       1938        348
Kelsey            89        124         38
Lucia              7         15          3
Holly             36         67         13
Elizabeth        124        181         44
Mikkel            11         18          5
Brandon           17         18          5
Jessie             5          5          2
Mike               1          1          1
Chen             168        242         53
Kai              212        381         86
Nancy             51        108         21
